<a href="https://colab.research.google.com/github/Hash-2003/TorrentToDrive/blob/main/TorrentToDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from pathlib import Path

BASE_DIR = Path("/content/drive/MyDrive/colab_torrent_projects")
JOB_NAME = "FOlder_Name" #folder name (Change this for every new torrent download)

JOB_DIR = BASE_DIR / JOB_NAME
DATA_DIR = JOB_DIR / "data"
STATE_DIR = JOB_DIR / "state"
LOG_DIR = JOB_DIR / "logs"

for d in [DATA_DIR, STATE_DIR, LOG_DIR]:
    d.mkdir(parents=True, exist_ok=True)

JOB_DIR, DATA_DIR, STATE_DIR, LOG_DIR

In [ ]:
!command -v aria2c >/dev/null || (apt-get update -qq && apt-get install -y aria2)
!aria2c -v


In [ ]:
import urllib.request
from pathlib import Path

TORRENT_LINK = (
    "PasteHere"  #link of the .torrent file (not the magnet)
)

TORRENT_FILE = STATE_DIR / "downloaded.torrent"
ARIA2_SESSION = STATE_DIR / "aria2.session"
ARIA2_LOG = LOG_DIR / "aria2.log"

print("TORRENT_FILE:", TORRENT_FILE)
print("ARIA2_SESSION:", ARIA2_SESSION)
print("ARIA2_LOG:", ARIA2_LOG)

if not TORRENT_FILE.exists():
    print("Downloading  torrent file...")
    urllib.request.urlretrieve(TORRENT_LINK, TORRENT_FILE)
else:
    print("Torrent file already exists.")

In [ ]:
from pathlib import Path

Path(ARIA2_SESSION).touch(exist_ok=True)
session_has_entries = ARIA2_SESSION.read_text(errors="ignore").strip()

if session_has_entries:
    print("Resuming from session file...")
    cmd = rf"""
    aria2c \
      --continue=true \
      --seed-time=0 \
      --enable-dht=true \
      --enable-dht6=false \
      --bt-enable-lpd=true \
      --bt-max-peers=55 \
      --split=16 \
      --min-split-size=5M \
      --piece-length=1M \
      --file-allocation=none \
      --dir="{DATA_DIR}" \
      --input-file="{ARIA2_SESSION}" \
      --save-session="{ARIA2_SESSION}" \
      --save-session-interval=60 \
      --log="{ARIA2_LOG}" \
      --log-level=notice
    """
else:
    print("Starting fresh from torrent...")
    cmd = rf"""
    aria2c \
      --continue=true \
      --seed-time=0 \
      --enable-dht=true \
      --enable-dht6=false \
      --bt-enable-lpd=true \
      --bt-max-peers=55 \
      --split=16 \
      --min-split-size=5M \
      --piece-length=1M \
      --file-allocation=none \
      --dir="{DATA_DIR}" \
      --save-session="{ARIA2_SESSION}" \
      --save-session-interval=60 \
      --log="{ARIA2_LOG}" \
      --log-level=notice \
      "{TORRENT_FILE}"
    """

!bash -lc "$cmd"

!ls -lh "{DATA_DIR}"
!tail -n 40 "{ARIA2_LOG}"

